In [1]:
import os
import re
import csv

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
import numpy as np
import math


In [2]:
directory = "20230727"
p = re.compile(r"\d\d\d")
instance_size_regex = re.compile(r"\d\d\d\d")

In [3]:
density_structure = {}
type_structure = {}

GROUNDING_TIME = "grounding_time"
GROUNDING_SIZE = "grounding_size"
TOTAL_TIME = "total_time"

GRINGO = "gringo"
IDLV = "idlv"
NEWGROUND_GRINGO = "newground-gringo"
NEWGROUND_IDLV = "newground-idlv"


grounder_types = [GRINGO, IDLV, NEWGROUND_GRINGO, NEWGROUND_IDLV]


file_types = [GROUNDING_TIME, GROUNDING_SIZE, TOTAL_TIME]


In [4]:
label_converter = {}
label_converter[GRINGO] = GRINGO
label_converter[IDLV] = IDLV
label_converter[NEWGROUND_GRINGO] = "NaGG-gringo"
label_converter[NEWGROUND_IDLV] = "NaGG-idlv"

In [5]:
info = {}
info[GROUNDING_SIZE] = {}
info[GROUNDING_SIZE]["filename"] = "00_benchmark_output_sm_aspcomp2013_grounding_size.csv" 
info[GROUNDING_SIZE]["title"] = "Grounding Size Plot"
info[GROUNDING_SIZE]["y_label"] = "Grounding Size [MB]"
info[GROUNDING_SIZE]["plot_file_name"] = "S1_stable_matching_grounding_size.svg" 



info[GROUNDING_TIME] = {}
info[GROUNDING_TIME]["filename"] = "00_benchmark_output_sm_aspcomp2013_grounding_time.csv" 
info[GROUNDING_TIME]["title"] = "Grounding Time Plot"
info[GROUNDING_TIME]["y_label"] = "Grounding Time [s]"
info[GROUNDING_TIME]["plot_file_name"] = "S1_stable_matching_grounding_time.svg" 




info[TOTAL_TIME] = {}
info[TOTAL_TIME]["filename"] = "00_benchmark_output_sm_aspcomp2013_total_time.csv" 
info[TOTAL_TIME]["title"] = "Total Time Plot"
info[TOTAL_TIME]["y_label"] = "Combined Time [s]"
info[TOTAL_TIME]["plot_file_name"] = "S1_stable_matching_total_time.svg" 


In [6]:
for key in info.keys():
    path = info[key]["filename"]
    info[key]["df"] = pd.read_csv(os.path.join(directory,path), delimiter=',')


In [57]:
# plot lines
for key in info.keys():
    
    plotting_columns = []
    
    for column in info[key]["df"].columns:
        if "timeout" not in column and "instance" not in column:              
            plotting_columns.append(column)
    
    list_instances = list(info[key]["df"]["instance"])
    instance_size_list = [int((instance_size_regex.search(instance)).group(0)) for instance in list_instances]

    for column in plotting_columns:
        
        label = None
        for strategy in grounder_types:
            if column.startswith(strategy):
                label = label_converter[strategy]
                
        if "size" in column:
            #plt.yscale('log')
            plot_column = [float(item)/1024 if float(item) <= 32*(math.pow(10,6)) else 32*(math.pow(10,6)) for item in list(info[key]["df"][column])]
        else:
            plot_column = list(info[key]["df"][column])
            
        plt.plot(instance_size_list, plot_column, label=label)
    
    plt.ylabel(info[key]["y_label"])
    plt.xlabel("Instance Number")
    #plt.title(info[key]["title"])
    
    handles, labels = plt.gca().get_legend_handles_labels()
    
    order = [2,3,1,0]
    
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],prop={'size': 14},loc='upper left')
    font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 14}


    plt.rc('font', **font)
    
    plt.show()
    
    #plt.savefig(info[key]["plot_file_name"], bbox_inches='tight', transparent=True)
    plt.close('all')